In [28]:
import pandas as pd
import numpy as np
DATA_PATH = '/Users/andreas/NHH/tech2-local/TECH2-H24/data/FRED'

# Exercise: Business cycle correlations

For this exercise, you'll be using macroeconomic data from the folder `../data/FRED`.

1.  There are seven decade-specific files named `FRED_monthly_19X0.csv` where `X` identifies the decade (`X` takes on the values 5, 6, 7, 8, 9, 0, 1). Write a loop that reads in all seven files as DataFrames and store them in a list.

    *Hint:* Recall from the lecture that you should use <TT>pd.read_csv(..., parse_dates=['DATE'])</TT> to automatically parse strings stored in the <TT>DATE</TT> column as dates.
2.  Use [`pd.concat()`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) to concate these data sets into a single `DataFrame` and set the `DATE` column as the index.
3.  You realize that your data does not include GDP since this variable is only reported at quarterly frequency.
    Load the GDP data from the file `GDP.csv` and merge it with your monthly data using an _inner join_.
4.  You want to compute how (percent) changes of the variables in your data correlate with percent changes in GDP.

    1. Create a _new_ `DataFrame` which contains the percent changes in CPI and GDP (using 
    [`pct_change()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pct_change.html), 
    see also the last exercise in workshop 3),
    and the absolute changes for the remaining variables (using 
    [`diff()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.diff.html)).
    2.  Compute the correlation of the percent changes in GDP with the (percent) changes of all other variables (using [`corr()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html)). What does the sign and magnitude of the correlation coefficient tell you?



In [29]:
# 1 , 2
list = []
for X in ['5','6','7','8','9','0','1']:
    if X == '0' or X== '1':
        list.append(pd.read_csv(f'{DATA_PATH}/FRED_monthly_20{X}0.csv', parse_dates=['DATE']))
    else:
        list.append(pd.read_csv(f'{DATA_PATH}/FRED_monthly_19{X}0.csv', parse_dates=['DATE']))
 
df =  pd.concat(list, axis = 0).reset_index(drop =True)
df


,DATE,CPI,UNRATE,FEDFUNDS,REALRATE,LFPART
0,1950-01-01,23.5,6.5,NaN,NaN,58.9
1,1950-02-01,23.6,6.4,NaN,NaN,58.9
2,1950-03-01,23.6,6.3,NaN,NaN,58.8
3,1950-04-01,23.6,5.8,NaN,NaN,59.2
4,1950-05-01,23.8,5.5,NaN,NaN,59.1
...,...,...,...,...,...,...
835,2019-08-01,256.0,3.6,2.1,0.6,63.1
836,2019-09-01,256.4,3.5,2.0,0.3,63.2
837,2019-10-01,257.2,3.6,1.8,-0.0,63.3
838,2019-11-01,257.9,3.6,1.6,-0.2,63.3


In [30]:
# 3
dfG = pd.read_csv(f'{DATA_PATH}/GDP.csv', parse_dates=['DATE'])
df =  pd.merge(df, dfG, on='DATE', how="inner")
df

,DATE,CPI,UNRATE,FEDFUNDS,REALRATE,LFPART,GDP
0,1950-01-01,23.5,6.5,NaN,NaN,58.9,2346.1
1,1950-04-01,23.6,5.8,NaN,NaN,59.2,2417.7
2,1950-07-01,24.1,5.0,NaN,NaN,59.1,2511.1
3,1950-10-01,24.5,4.2,NaN,NaN,59.4,2559.2
4,1951-01-01,25.4,3.7,NaN,NaN,59.1,2594.0
...,...,...,...,...,...,...,...
275,2018-10-01,252.8,3.8,2.2,-0.2,62.9,20304.9
276,2019-01-01,252.6,4.0,2.4,0.6,63.1,20431.6
277,2019-04-01,255.2,3.7,2.4,3.1,62.8,20602.3
278,2019-07-01,255.8,3.7,2.4,1.1,63.1,20843.3


In [31]:
# 4
dg = df[['DATE','CPI', 'GDP']].set_index('DATE')
dg = dg.pct_change(periods =3)*100


In [32]:
columns = ['UNRATE','FEDFUNDS','REALRATE','LFPART']
dg[columns] = df[columns].diff()
dg

,CPI,GDP,UNRATE,FEDFUNDS,REALRATE,LFPART
DATE,,,,,,
1950-01-01,NaN,NaN,NaN,NaN,NaN,NaN
1950-04-01,NaN,NaN,NaN,NaN,NaN,NaN
1950-07-01,NaN,NaN,NaN,NaN,NaN,NaN
1950-10-01,4.255319,9.083159,NaN,NaN,NaN,NaN
1951-01-01,7.627119,7.292054,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2018-10-01,1.566894,1.301131,NaN,NaN,NaN,NaN
2019-01-01,0.959233,1.395003,NaN,NaN,NaN,NaN
2019-04-01,1.592357,1.608290,NaN,NaN,NaN,NaN


***
# Exercise: Loading many data files

In the previous exercise, you loaded the individual files by specifing an explicit list of file names. This can become tedious or infeasible if your data is spread across many files with varying file name patterns. Python offers the possibility to iterate over all files in a directory (for example, using [`os.listdir()`](https://docs.python.org/3/library/os.html#os.listdir)),
or to iterate over files that match a pattern, for example using [`glob.glob()`](https://docs.python.org/3/library/glob.html).

Repeat parts (1) and (2) from the previous exercise, but now iterate over the input files using 
[`glob.glob()`](https://docs.python.org/3/library/glob.html). You'll need to use a wildcard `*` and make sure to match only the relevant files in `../data/FRED`, i.e., those that start with `FRED_monthly`.

In [33]:
import glob
data_list = []

files = glob.glob('/Users/andreas/NHH/tech2-local/TECH2-H24/data/FRED/FRED_monthly_*.csv')
for file in files:
    data_list.append(pd.read_csv(file, parse_dates=['DATE']))

df = pd.concat(data_list).set_index('DATE').sort_index()
df


,CPI,UNRATE,FEDFUNDS,REALRATE,LFPART,INFLATION
DATE,,,,,,
1947-01-01,21.5,NaN,NaN,NaN,NaN,NaN
1947-02-01,21.6,NaN,NaN,NaN,NaN,NaN
1947-03-01,22.0,NaN,NaN,NaN,NaN,NaN
1947-04-01,22.0,NaN,NaN,NaN,NaN,NaN
1947-05-01,22.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2024-06-01,313.0,4.1,5.3,2.3,62.6,3.0
2024-07-01,313.5,4.3,5.3,2.5,62.7,2.9
2024-08-01,314.1,4.2,5.3,2.5,62.7,2.6


***
# Exercise: Decade averages of macro time series


For this exercise, you'll be using macroeconomic data from the folder `../data/FRED`.

1.  There are five files containing monthly observations on annual inflation (INFLATION), the Fed Funds rate (FEDFUNDS), the labor force participation rate (LFPART), the 1-year real interest rate (REALRATE) and the unemployment rate (UNRATE).
    Write a loop to import these and merge them on `DATE` into a single `DataFrame` using _outer joins_ (recall that [`merge()`](https://pandas.pydata.org/docs/reference/api/pandas.merge.html) 
    and [`join()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html) operate on only two DataFrames at a time). 

    *Hint:* Recall from the lecture that you should use <TT>pd.read_csv(..., parse_dates=['DATE'])</TT> to automatically parse strings stored in the <TT>DATE</TT> column as dates.

2.  Your friend is a pandas guru and tells you that you don't need to iteratively merge many files but can instead directly use [`pd.concat()`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) for merging many DataFrames in a single step.
    Repeat the previous part using `pd.concat()` instead, and verify that you get the same result (you can do this using [`compare()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.compare.html)).
3.  You want to compute the average value of each variable by decade, but you want to include only decades without _any_ missing values for _all_ variables.
    1.  Create a variable `Decade` which stores the decade (1940, 1950, ...) for each observation.

        *Hint:* You should have set the `DATE` as the `DataFrame` index. Then you can access the calendar year using the attribute `df.index.year` which can be used to compute the decade.
    2.  Write a function `num_missing(x)` which takes as argument `x` a `Series` and returns the number of missing values in this `Series`.
    3.  Compute the number of missing values by decade for each variable using a [`groupby()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) operation and the function `num_missing` you wrote.
    4.  Aggregate this data across all variables to create an indicator for each decade whether there are any missing values. This can be done in many ways but will require aggregation across columns, e.g., with `sum(..., axis=1)`.
    5.  Merge this decade-level indicator data back into the original `DataFrame` (_many-to-one_ merge). 
4.  Using this indicator, drop all observations which are in a decade with missing values.
5.  Compute the decade average for each variable.

**Challenge**

-   Your pandas guru friend claims that all the steps in 3.2 to 3.5 can be done with a single one-liner using [`transform()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.transform.html). Can you come up with a solution?

    

In [34]:
# 1
df = pd.read_csv(f'{DATA_PATH}/INFLATION.csv', parse_dates=['DATE']).set_index('DATE')

for i in ['FEDFUNDS','LFPART','REALRATE','UNRATE']:
    df1 = (pd.read_csv(f'{DATA_PATH}/{i}.csv', parse_dates=['DATE']).set_index('DATE'))
    df = df.merge(df1, on = 'DATE', how ='outer')
df

,INFLATION,FEDFUNDS,LFPART,REALRATE,UNRATE
DATE,,,,,
1948-01-01,10.2,NaN,58.6,NaN,3.4
1948-02-01,9.5,NaN,58.9,NaN,3.8
1948-03-01,6.8,NaN,58.5,NaN,4.0
1948-04-01,8.3,NaN,59.0,NaN,3.9
1948-05-01,9.4,NaN,58.3,NaN,3.5
...,...,...,...,...,...
2024-06-01,3.0,5.3,62.6,2.3,4.1
2024-07-01,2.9,5.3,62.7,2.5,4.3
2024-08-01,2.6,5.3,62.7,2.5,4.2


In [35]:
# 2
list = []
for i in ['INFLATION','FEDFUNDS','LFPART','REALRATE','UNRATE']:
    list.append(pd.read_csv(f'{DATA_PATH}/{i}.csv', parse_dates=['DATE']).set_index('DATE'))

df2 = pd.concat(list, axis = 1, join='outer')
df2

,INFLATION,FEDFUNDS,LFPART,REALRATE,UNRATE
DATE,,,,,
1948-01-01,10.2,NaN,58.6,NaN,3.4
1948-02-01,9.5,NaN,58.9,NaN,3.8
1948-03-01,6.8,NaN,58.5,NaN,4.0
1948-04-01,8.3,NaN,59.0,NaN,3.9
1948-05-01,9.4,NaN,58.3,NaN,3.5
...,...,...,...,...,...
2024-06-01,3.0,5.3,62.6,2.3,4.1
2024-07-01,2.9,5.3,62.7,2.5,4.3
2024-08-01,2.6,5.3,62.7,2.5,4.2


In [36]:
pd.DataFrame.compare(df,df2)

DATE


In [37]:
# Alternative way: 
import os.path

data = []

variables = ['INFLATION','FEDFUNDS','LFPART','REALRATE','UNRATE']

for var in variables:
    fn = os.path.join(DATA_PATH, f'{var}.csv')
    data.append(pd.read_csv(fn, parse_dates=['DATE'], index_col='DATE'))

df = pd.concat(data, axis=1, join='outer')
df

,INFLATION,FEDFUNDS,LFPART,REALRATE,UNRATE
DATE,,,,,
1948-01-01,10.2,NaN,58.6,NaN,3.4
1948-02-01,9.5,NaN,58.9,NaN,3.8
1948-03-01,6.8,NaN,58.5,NaN,4.0
1948-04-01,8.3,NaN,59.0,NaN,3.9
1948-05-01,9.4,NaN,58.3,NaN,3.5
...,...,...,...,...,...
2024-06-01,3.0,5.3,62.6,2.3,4.1
2024-07-01,2.9,5.3,62.7,2.5,4.3
2024-08-01,2.6,5.3,62.7,2.5,4.2



3.  You want to compute the average value of each variable by decade, but you want to include only decades without _any_ missing values for _all_ variables.
    1.  Create a variable `Decade` which stores the decade (1940, 1950, ...) for each observation.

        *Hint:* You should have set the `DATE` as the `DataFrame` index. Then you can access the calendar year using the attribute `df.index.year` which can be used to compute the decade.
    2.  Write a function `num_missing(x)` which takes as argument `x` a `Series` and returns the number of missing values in this `Series`.
    3.  Compute the number of missing values by decade for each variable using a [`groupby()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) operation and the function `num_missing` you wrote.
    4.  Aggregate this data across all variables to create an indicator for each decade whether there are any missing values. This can be done in many ways but will require aggregation across columns, e.g., with `sum(..., axis=1)`.
    5.  Merge this decade-level indicator data back into the original `DataFrame` (_many-to-one_ merge). 

In [38]:
# Extract calendar year()
year = df.index.year

decade = (year // 10)* 10
df['Decade'] = decade

df['Decade'].value_counts()


Decade
1950    120
1960    120
1970    120
1980    120
1990    120
2000    120
2010    120
2020     58
1940     24
Name: count, dtype: int64

In [46]:
# Challenge
df['NotMissing'] = df.groupby('Decade').transform(lambda x: x.isna().sum()).sum(axis=1) == 0
df

,INFLATION,FEDFUNDS,LFPART,REALRATE,UNRATE,Decade,NotMissing
DATE,,,,,,,
1948-01-01,10.2,NaN,58.6,NaN,3.4,1940,False
1948-02-01,9.5,NaN,58.9,NaN,3.8,1940,False
1948-03-01,6.8,NaN,58.5,NaN,4.0,1940,False
1948-04-01,8.3,NaN,59.0,NaN,3.9,1940,False
1948-05-01,9.4,NaN,58.3,NaN,3.5,1940,False
...,...,...,...,...,...,...,...
2024-06-01,3.0,5.3,62.6,2.3,4.1,2020,False
2024-07-01,2.9,5.3,62.7,2.5,4.3,2020,False
2024-08-01,2.6,5.3,62.7,2.5,4.2,2020,False


In [12]:
x = df['Decade']
def num_missing(x): 
    n = x.isna().sum()
    return n


In [40]:
df_miss = df.groupby('Decade').agg(num_missing)
df_miss

,INFLATION,FEDFUNDS,LFPART,REALRATE,UNRATE
Decade,,,,,
1940,0,24,0,24,0
1950,0,54,0,120,0
1960,0,0,0,120,0
1970,0,0,0,120,0
1980,0,0,0,24,0
1990,0,0,0,0,0
2000,0,0,0,0,0
2010,0,0,0,0,0
2020,2,1,1,0,1


In [14]:
no_miss = (df_miss.sum(axis=1))==0
no_miss

no_miss = no_miss.to_frame('NotMissing')
no_miss = no_miss.reset_index()
no_miss

,Decade,NotMissing
0,1940,False
1,1950,False
2,1960,False
3,1970,False
4,1980,False
5,1990,True
6,2000,True
7,2010,True
8,2020,False


In [15]:
df = df.merge(no_miss, on='Decade', how ='left')
df.head(5)

,INFLATION,FEDFUNDS,LFPART,REALRATE,UNRATE,Decade,NotMissing
0,10.2,NaN,58.6,NaN,3.4,1940,False
1,9.5,NaN,58.9,NaN,3.8,1940,False
2,6.8,NaN,58.5,NaN,4.0,1940,False
3,8.3,NaN,59.0,NaN,3.9,1940,False
4,9.4,NaN,58.3,NaN,3.5,1940,False


,INFLATION,FEDFUNDS,LFPART,REALRATE,UNRATE,Decade,NotMissing
0,10.2,NaN,58.6,NaN,3.4,1940,False
1,9.5,NaN,58.9,NaN,3.8,1940,False
2,6.8,NaN,58.5,NaN,4.0,1940,False
3,8.3,NaN,59.0,NaN,3.9,1940,False
4,9.4,NaN,58.3,NaN,3.5,1940,False
...,...,...,...,...,...,...,...
917,3.0,5.3,62.6,2.3,4.1,2020,False
918,2.9,5.3,62.7,2.5,4.3,2020,False
919,2.6,5.3,62.7,2.5,4.2,2020,False
920,NaN,5.1,62.7,2.4,4.1,2020,False


***
# Exercise: Mering the Titanic data

In this exercise, you'll be working with the the original Titanic data set in `titanic.csv` and additional (partly fictitious) information on passengers stored in `titanic-additional.csv`, both located in the `data/` folder.

The goal of the exercise is to calculate the survival rates by country of residence (for this exercise we restrict ourselves to the UK, so these will be England, Scotland, etc.).

1.  Load the `titanic.csv` and `titanic-additional.csv` into two DataFrames.

    Inspect the columns contained in both data sets. As you can see, the original data contains the full name including the title
    and potentially maiden name (for married women) in a single column.
    The additional data contains this information in separate columns.
    You want to merge these data sets, but you first need to create common keys in both DataFrames.

2.  Since the only common information is the name, you'll need to extract the individual name components from the original DataFrame
    and use these as merge keys.

    Focusing only on men (who have names that are much easier to parse), split the `Name` column into the tokens 
    `Title`, `FirstName` and `LastName`, just like the columns in the second DataFrame.

    *Hint:* This is the same task as in the last exercise in Workshop 2. You can just use your solution here.

3.  Merge the two data sets based on the columns `Title`, `FirstName` and `LastName` you just created using a _left join_ (_one-to-one_ merge).
    Tabulate the columns and the number of non-missing observations to make sure that merging worked. 

    *Note:* The additional data set contains address information only for passengers from the UK, so some of these fields will be missing.

4.  You are now in a position to merge the country of residence (_many-to-one_ merge). Load the country data from `UK_post_codes.csv` which contains 
    the UK post code prefix (which you can ignore), the corresponding city, and the corresponding country.

    Merge this data with your passenger data set using a _left join_ (what is the correct merge key?).

5.  Tabulate the number of observations by `Country`, including the number of observations with missing `Country` (these are passengers residing outside the UK).

    Finally, compute the mean survival rate by country.

In [557]:
import glob

DATA_PATH = '/Users/andreas/NHH/tech2-local/TECH2-H24/data'

df1 = pd.read_csv(f'{DATA_PATH}/titanic.csv')
df1.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss Laina",female,26.0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,373450,8.0500,NaN,S


In [558]:
df2 = pd.read_csv(f'{DATA_PATH}/titanic-additional.csv')
df2.head(5)

,Title,LastName,FirstName,MaidenName,City,Postcode,Address
0,Mr.,Christmann,Emil,NaN,Chester,CH6 34H,3 Graham ways
1,Miss,Heikkinen,Laina,NaN,Bolton,BL0 1XG,0 Griffin wells
2,Lady.,Duff Gordon,Lucille Christiana,Sutherland,NaN,NaN,NaN
3,Miss,Pettersson,Ellen Natalia,NaN,Northampton,NN0 H5R,889 Murray glen
4,Mr.,Odahl,Nils Martin,NaN,Derby,DE7 QZ7,"Studio 2, Long courts"


In [559]:
df1 = df1[df1['Sex'] == 'male']
names = df1['Name'].str.partition(',')
names2 = names[2].str.strip().str.partition(' ')


df1['FirstName'] = names2[2]
df1['Title'] = names2[0]
df1['LastName'] = names[0]
del df1['Name']
df1.head(5)

,PassengerId,Survived,Pclass,Sex,Age,Ticket,Fare,Cabin,Embarked,FirstName,Title,LastName
0,1,0,3,male,22.0,A/5 21171,7.2500,NaN,S,Owen Harris,Mr.,Braund
4,5,0,3,male,35.0,373450,8.0500,NaN,S,William Henry,Mr.,Allen
5,6,0,3,male,NaN,330877,8.4583,NaN,Q,James,Mr.,Moran
6,7,0,1,male,54.0,17463,51.8625,E46,S,Timothy J,Mr.,McCarthy
7,8,0,3,male,2.0,349909,21.0750,NaN,S,Gosta Leonard,Master,Palsson


In [560]:
df = pd.merge(df1,df2, on =['Title', "LastName", 'FirstName'], how ='left')
df.info()
df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 577 entries, 0 to 576
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  577 non-null    int64  
 1   Survived     577 non-null    int64  
 2   Pclass       577 non-null    int64  
 3   Sex          577 non-null    object 
 4   Age          453 non-null    float64
 5   Ticket       577 non-null    object 
 6   Fare         577 non-null    float64
 7   Cabin        107 non-null    object 
 8   Embarked     577 non-null    object 
 9   FirstName    577 non-null    object 
 10  Title        577 non-null    object 
 11  LastName     577 non-null    object 
 12  MaidenName   0 non-null      object 
 13  City         471 non-null    object 
 14  Postcode     471 non-null    object 
 15  Address      471 non-null    object 
dtypes: float64(2), int64(3), object(11)
memory usage: 72.3+ KB


,PassengerId,Survived,Pclass,Sex,Age,Ticket,Fare,Cabin,Embarked,FirstName,Title,LastName,MaidenName,City,Postcode,Address
0,1,0,3,male,22.0,A/5 21171,7.2500,NaN,S,Owen Harris,Mr.,Braund,NaN,Portsmouth,PO9 1CI,"Flat 6, Dean well"
1,5,0,3,male,35.0,373450,8.0500,NaN,S,William Henry,Mr.,Allen,NaN,Chelmsford,CM3 V66,"Studio 65, Leah falls"
2,6,0,3,male,NaN,330877,8.4583,NaN,Q,James,Mr.,Moran,NaN,Southall,UB8 8SZ,9 Teresa lakes
3,7,0,1,male,54.0,17463,51.8625,E46,S,Timothy J,Mr.,McCarthy,NaN,Wakefield,WF0 GXT,"Studio 7, Mark dale"
4,8,0,3,male,2.0,349909,21.0750,NaN,S,Gosta Leonard,Master,Palsson,NaN,London,W55 LK3,"Studio 11B, Paula lane"


In [562]:
fn = os.path.join(DATA_PATH, 'UK_post_codes.csv')
df3 = pd.read_csv(fn)
del df3['Prefix']
df3

,City,Country
0,Aberdeen,Scotland
1,St Albans,England
2,Birmingham,England
3,Bath,England
4,Blackburn,England
...,...,...
115,Worcester,England
116,Walsall,England
117,Wolverhampton,England
118,York,England


In [563]:
df = df.merge(df3, on ='City', how='left')
df

,PassengerId,Survived,Pclass,Sex,Age,Ticket,Fare,Cabin,Embarked,FirstName,Title,LastName,MaidenName,City,Postcode,Address,Country
0,1,0,3,male,22.0,A/5 21171,7.2500,NaN,S,Owen Harris,Mr.,Braund,NaN,Portsmouth,PO9 1CI,"Flat 6, Dean well",England
1,5,0,3,male,35.0,373450,8.0500,NaN,S,William Henry,Mr.,Allen,NaN,Chelmsford,CM3 V66,"Studio 65, Leah falls",England
2,6,0,3,male,NaN,330877,8.4583,NaN,Q,James,Mr.,Moran,NaN,Southall,UB8 8SZ,9 Teresa lakes,England
3,7,0,1,male,54.0,17463,51.8625,E46,S,Timothy J,Mr.,McCarthy,NaN,Wakefield,WF0 GXT,"Studio 7, Mark dale",England
4,8,0,3,male,2.0,349909,21.0750,NaN,S,Gosta Leonard,Master,Palsson,NaN,London,W55 LK3,"Studio 11B, Paula lane",England
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,884,0,2,male,28.0,C.A./SOTON 34068,10.5000,NaN,S,Frederick James,Mr.,Banfield,NaN,Lancaster,LA0 PNF,"Studio 1, Gavin ports",England
776,885,0,3,male,25.0,SOTON/OQ 392076,7.0500,NaN,S,Henry Jr,Mr.,Sutehall,NaN,Kilmarnock,KA1 NO8,"Studio 55b, Smith village",Scotland
777,887,0,2,male,27.0,211536,13.0000,NaN,S,Juozas,Rev.,Montvila,NaN,Croydon,CR1 NLI,041 Jackson passage,England
778,890,1,1,male,26.0,111369,30.0000,C148,C,Karl Howell,Mr.,Behr,NaN,NaN,NaN,NaN,NaN


In [568]:
df['Country'].value_counts(dropna=False)

Country
England             601
NaN                 106
Scotland             67
Northern Ireland      6
Name: count, dtype: int64

In [574]:
# mean survival rate by country

group = df.groupby('Country', dropna=False)['Survived'].mean()*100
group

Country
England             11.480865
Northern Ireland     0.000000
Scotland            17.910448
NaN                 33.018868
Name: Survived, dtype: float64